# Grouping Nodes <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/grouping.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook showcases the `add_parent_configuration` method, which allows to configure relationships as grouping hierarchy.

Relationships that are used as `parent` configuration are removed from the visualization and instead, the source-node is created as child node of the relationship's target.

In [1]:
%pip install yfiles_jupyter_graphs_for_neo4j --quiet
%pip install neo4j --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 1.1 MB/s eta 0:00:00


You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [2]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

<a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/grouping.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from neo4j import GraphDatabase

NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com"
NEO4J_USERNAME = "fincen"
NEO4J_PASSWORD = "fincen"
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'fincen')

In [4]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

# pass the Neo4j database driver to resolve the queries
g = Neo4jGraphWidget(driver)
# automatically load relationships in the returned node collection of the cypher queries
g.set_autocomplete_relationships(True)

The following sample shows the graph of five countries that have at least five connected entities.

In [5]:
country_relations_cypher = """MATCH (entity:Entity)-[r:COUNTRY]->(country:Country)
            WHERE country.name IN ['China', 'Denmark', 'Ecuador', 'France', 'Germany'] WITH country, collect(entity) AS entities
            WHERE size(entities) >= 5 WITH country, entities[0..5] AS limitedEntities RETURN country, limitedEntities LIMIT 5"""
g.show_cypher(country_relations_cypher)

GraphWidget(layout=Layout(height='800px', width='100%'))

## Configure the `country` relationship as nested hierarchy

Specifying a relationship as parent configuration replaces these edges in the visualization with a nested parent-child structure.

In [6]:
# use "COUNTRY" relationships as nested hierarchies
g.add_parent_relationship_configuration('COUNTRY')

# the 'Country' nodes now act as group nodes which can also be configured
g.add_node_configuration('Country', text='code')

g.show_cypher(country_relations_cypher)

GraphWidget(layout=Layout(height='800px', width='100%'))

## Configure new artificial groups

It is also possible to configure artificial groups that are not within the database. For example, we'll group add group nodes for the different node labels:

In [7]:
g = Neo4jGraphWidget(driver)
g.set_autocomplete_relationships(True)

# creates a new group node for nodes with label 'Entity'
g.add_node_configuration('Entity', parent_configuration='Entities')

# creates a new *blue* group node for nodes with label 'Country' and
g.add_node_configuration('Country', parent_configuration={'text': 'Countries', 'color': '#2196F3'})

g.show_cypher(country_relations_cypher)

GraphWidget(layout=Layout(height='800px', width='100%'))

## Multi-level nesting

Each node can have exactly one parent node. However, if you set a parent node for this node as well, you can achieve multi-level nesting

In [8]:
g = Neo4jGraphWidget(driver)
g.set_autocomplete_relationships(True)
# a query that returns a graph of three chained nodes
chained_relationship_query = """
MATCH (f:Filing {begin_date: "Mar 9, 2016", end_date: "Aug 10, 2016"})
WITH f
MATCH (e:Entity {name: "Macquarie Bank Ltd", country: "AUS"})
WITH f, e
MATCH (c:Country {code: "AUS"})
RETURN f, e, c
"""
g.show_cypher(chained_relationship_query)

GraphWidget(layout=Layout(height='500px', width='100%'))

### Adding multiple parent relationships

In [9]:
g.add_parent_relationship_configuration("CONCERNS")
g.add_parent_relationship_configuration("COUNTRY")
g.show_cypher(chained_relationship_query)

GraphWidget(layout=Layout(height='500px', width='100%'))

### Removing parent relationships

In [10]:
g.del_parent_relationship_configuration("COUNTRY")
g.del_parent_relationship_configuration("CONCERNS")
g.show_cypher(chained_relationship_query)

GraphWidget(layout=Layout(height='500px', width='100%'))